In [1]:
# load module and pre generate data
from graph_tree_module import Node, sim, graph_tree

import numpy as np

import pickle

# Open the file in binary read mode and load the object
with open("graph_tree.pkl", "rb") as file:
    gt = pickle.load(file)


In [2]:
sliders_data = [{"id": gt.attr_name[i], "min": gt.X_maxmin[1][i], "max": gt.X_maxmin[0][i], "value": gt.root.center[i], "type": "f"} for i in range(len(gt.attr_name))]
# print(sliders_data)

recom_data = [{"i": i, "id": gt.attr_name[i], "max_val": gt.recom[i][0].center[i], "min_val": gt.recom[i][1].center[i]} for i in range(len(gt.attr_name))]
# print(recom_data)

status = gt.get_suround_cluster(gt.root, gt.root.center)
print(status)
def get_suround_node_data(status):
    cur_node = status["cur_cluster"]
    cur_cluster = [{"id": cur_node.id, "position": "center", "x": 300, "y": 200, "color": "red"}]

    contra_node = status["contra_cluster"]
    contra_cluster = [{"id": contra_node[i].id, "position": "left", "x": 100, "y": 150+i*100, "color": "green"} for i in range(len(contra_node))]

    super_node = status["super_cluster"]
    super_cluster = [{"id": super_node.id, "position": "top", "x": 300, "y": 100, "color": "dark red"}] if super_node else []

    sim_node = status["neibour_cluster"]
    sim_cluster = [{"id": sim_node[i].id, "position": "right", "x": 500, "y": 200+i*100, "color": "blue"} for i in range(len(sim_node))] if sim_node else []

    sub_node = status["sub_cluster"]
    sub_cluster = [{"id": sub_node[i].id, "position": "bottom", "x": 200+i*100, "y": 350, "color": "pink"} for i in range(len(sub_node))] if sub_node else []

    return cur_cluster + contra_cluster + super_cluster + sim_cluster + sub_cluster

graph_data = get_suround_node_data(status)

print(graph_data)

def get_tree_data(status):
    cur_node = status["cur_cluster"]
    up1_node = cur_node.up_lv if cur_node.up_lv else []
    up2_node = up1_node.up_lv if up1_node.up_lv else []

tree_data = {
    'name': 'Root',
    'children': [
        {'name': 'Child 1', 'children': [
            {'name': 'Grandchild 1'},
            {'name': 'Grandchild 2'}
        ]},
        {'name': 'Child 2', 'children': [
            {'name': 'Grandchild 3'}
        ]}
    ]
}

[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.250018

In [3]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

# slider ####################################################################
@app.route('/sliders')
def sliders():
    return jsonify(sliders_data)

@app.route('/handle_slider', methods=['POST'])
def submit_slider_values():
    data = request.json
    print('Received slider values:', data)
    query = np.array([float(val) for val in data.values()])
    cur = gt.search(query)
    print(cur.id)
    print(gt.status)
    global graph_data
    graph_data = get_suround_node_data(gt.status)
    # Process the data as needed
    return jsonify({'status': 'slider respon success', 'received': data})
# slider end ################################################################

# recom ####################################################################
@app.route('/recom')
def recom():
    return jsonify(recom_data)

@app.route('/handle_recom', methods=['POST'])
def handle_recom():
    data = request.json
    button_id = int(data.get('buttonId'))
    val = int(data.get('val'))
    print(button_id, val)
    moved_node = gt.recom[button_id][val]
    gt.get_suround_cluster(moved_node)
    global graph_data
    graph_data = get_suround_node_data(gt.status)
    # Handle the button click based on the button_id
    return jsonify({'status': 'recom respon success', 'buttonId': button_id, 'val': val})
# recom end ####################################################################

@app.route('/graph-data')
def graph():
    return jsonify(graph_data)

@app.route('/tree-data')
def tree():
    return jsonify(tree_data)

if __name__ == '__main__':
    app.run(port=8123)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8123
Press CTRL+C to quit
127.0.0.1 - - [06/Sep/2024 11:56:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2024 11:56:15] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [06/Sep/2024 11:56:15] "GET /static/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [06/Sep/2024 11:56:15] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2024 11:56:15] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2024 11:56:15] "GET /graph-data HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2024 11:56:15] "GET /tree-data HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2024 11:56:23] "POST /handle_recom HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2024 11:56:23] "GET /graph-data HTTP/1.1" 200 -


13 0
[0.33925926 0.67237827 0.74295219 0.85636364 0.78706096 0.
 0.07104574 0.14717132 0.03056334 0.18666394 0.74485803 0.46749678
 0.14423913 0.75      ] [0.48842944 0.70722638 0.70365915 0.868045   0.78315387 0.
 0.10980067 0.15543423 0.01980725 0.16204362 0.70021365 0.45958429
 0.13929375 0.74832776]
[0.33925926 0.67237827 0.74295219 0.85636364 0.78706096 0.
 0.07104574 0.14717132 0.03056334 0.18666394 0.74485803 0.46749678
 0.14423913 0.75      ] [0.47678936 0.58652752 0.74931653 0.85786307 0.79420531 0.
 0.08568628 0.10873536 0.02346942 0.18740648 0.54260227 0.48446861
 0.14968165 0.74150268]
[0.33925926 0.67237827 0.74295219 0.85636364 0.78706096 0.
 0.07104574 0.14717132 0.03056334 0.18666394 0.74485803 0.46749678
 0.14423913 0.75      ] [0.48842944 0.70722638 0.70365915 0.868045   0.78315387 0.
 0.10980067 0.15543423 0.01980725 0.16204362 0.70021365 0.45958429
 0.13929375 0.74832776]
[0.33925926 0.67237827 0.74295219 0.85636364 0.78706096 0.
 0.07104574 0.14717132 0.03056334 0.

127.0.0.1 - - [06/Sep/2024 11:56:25] "POST /handle_recom HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2024 11:56:25] "GET /graph-data HTTP/1.1" 200 -


11 1
[0.49964433 0.47236941 0.76710786 0.77080666 0.78438528 1.
 0.03948051 0.0432285  0.0521637  0.12978865 0.33030379 0.39312799
 0.17792998 0.73591549] [0.46949495 0.48498585 0.66003496 0.70809917 0.76457573 1.
 0.0372945  0.09093549 0.03302536 0.142925   0.30104132 0.44749697
 0.16623631 0.72363636]
[0.49964433 0.47236941 0.76710786 0.77080666 0.78438528 1.
 0.03948051 0.0432285  0.0521637  0.12978865 0.33030379 0.39312799
 0.17792998 0.73591549] [0.45140012 0.42362339 0.77132408 0.71352313 0.79613696 1.
 0.05869357 0.06111105 0.02859918 0.2114266  0.32877594 0.53122751
 0.16505946 0.73131673]
[0.49964433 0.47236941 0.76710786 0.77080666 0.78438528 1.
 0.03948051 0.0432285  0.0521637  0.12978865 0.33030379 0.39312799
 0.17792998 0.73591549] [0.46949495 0.48498585 0.66003496 0.70809917 0.76457573 1.
 0.0372945  0.09093549 0.03302536 0.142925   0.30104132 0.44749697
 0.16623631 0.72363636]
[0.49964433 0.47236941 0.76710786 0.77080666 0.78438528 1.
 0.03948051 0.0432285  0.0521637  0.

127.0.0.1 - - [06/Sep/2024 11:56:26] "POST /handle_recom HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2024 11:56:26] "GET /graph-data HTTP/1.1" 200 -


9 1
[0.40246502 0.40305784 0.87915604 0.1501251  0.82056698 1.
 0.0535832  0.02086652 0.04925093 0.12001245 0.39491725 0.47098174
 0.19330571 0.7293578 ] [0.42049205 0.36889644 0.87812206 0.14131413 0.82424655 1.
 0.06669662 0.02125381 0.03491716 0.15596943 0.37353552 0.59415953
 0.17801735 0.73019802]
[0.40246502 0.40305784 0.87915604 0.1501251  0.82056698 1.
 0.0535832  0.02086652 0.04925093 0.12001245 0.39491725 0.47098174
 0.19330571 0.7293578 ] [0.40492028 0.39423608 0.88580769 0.14222363 0.82567834 1.
 0.07185549 0.02380941 0.03697019 0.24746831 0.37100766 0.5486018
 0.17287031 0.72996516]
[0.40246502 0.40305784 0.87915604 0.1501251  0.82056698 1.
 0.0535832  0.02086652 0.04925093 0.12001245 0.39491725 0.47098174
 0.19330571 0.7293578 ] [0.42049205 0.36889644 0.87812206 0.14131413 0.82424655 1.
 0.06669662 0.02125381 0.03491716 0.15596943 0.37353552 0.59415953
 0.17801735 0.73019802]
[0.40246502 0.40305784 0.87915604 0.1501251  0.82056698 1.
 0.0535832  0.02086652 0.04925093 0.12